In [2]:
import os 
#if you want to know current working dir
os.getcwd()
os.chdir('/home/jecosine/Courses/Courses/COMP8800/Projects/CatBoostPlayground')
# os.listdir()

from allib.datasets import load_uci, AVAIL_DATASETS
from allib.metrics import distance
from allib.utils import ensure_path
from allib.models.al import get_al_strategy
from sklearn.metrics.pairwise import pairwise_distances, check_pairwise_arrays
from allib.metrics import get_metrics
from allib.plots import PLMetric
from allib.models import get_pipeline, AVAIL_MODELS
import numpy as np
import pandas as pd
import os
import pickle
# os.environ["OMP_NUM_THREADS"] = "1"
# ignore numpy warning
import warnings
warnings.filterwarnings('ignore')

#### TODO:

- [x] add stratified sampling
- [ ] compare with random baseline

In [3]:
# DIST_CACHE = "../../examples/dist_cache_1000"
DIST_CACHE = "examples/dist_cache_1000"
LOCAL_DIR = "app/comp_dist"
# LOCAL_DIR = "./"
# ENCODE = "onehot-dense"
ENCODE = "ordinal"
ALSTRATEGY = "typiclust"

## GSX

In [ ]:
for dsn in AVAIL_DATASETS:
    ds = load_uci(dsn)
    ds.with_preprocess(steps=["sample_n", "continuous_to_categorical", "remove_constant_columns", "drop_duplicate_rows"],  params_list=[{"n": 1000, "random_state": 0}, {"encode": "ordinal"}, {}, {}], in_place=True)
    model_name = "catboost"
    gsx = get_al_strategy("gsx")
    for metric in distance.AVAIL_DIST_METRICS:
    # for metric in ["cosine", "euclidean"]:
        fn = f"{DIST_CACHE}/{dsn}/{metric}_ordinal.npy"
        cache_name = f"{dsn.replace('/', '_')}@{model_name}@gsx_{metric}@x20.pkl"
        if ensure_path(os.path.join("ppl_cache", cache_name), False):
            print(f"exp {cache_name} already exists, continue")
            continue
        if not ensure_path(fn, False):
            print(f"{metric} for {dsn} not found, skipping ...")
            continue
        print(f"Using cache of {metric} for {dsn}")
        setattr(gsx, "dist_cache_path", fn)
        ds = ds.with_strategy(gsx, extra_params={"distance_metric": metric})
        make_ppl = get_pipeline(model_name)
        ppl = make_ppl(
            model=None,
            eval_metrics=get_metrics(["accuracy"]),
            seeds=[i for i in range(20)],
            n_times=20,
            dataset=ds,
            cat_idx=ds.info["cat_idx"]
        )
        ppl.start()
        with open(os.path.join("ppl_cache", cache_name), "wb") as f:
            pickle.dump(ppl.stats, f)


In [80]:
ds = load_uci(dsn)    
ds.with_preprocess(steps=["sample_n", "continuous_to_categorical", "remove_constant_columns", "drop_duplicate_rows"],  params_list=[{"n": 1000, "random_state": 0}, {"encode": ENCODE}, {}, {}], in_place=True)
ds._data.reset_index(drop=True, inplace=True)

In [6]:
from allib.metrics.distance import AVAIL_DIST_METRICS 
from scipy.spatial.distance import squareform
for metric in AVAIL_DIST_METRICS:
    fn = f"{DIST_CACHE}/adult/{metric}_catonly.npy"
    # if ensure_path(fn, False):
    #     print(f"{metric} for adult already exists, continue")
    #     continue
    d = np.load(fn)
    # dd = squareform(d)
    print(d.shape)

(654, 654)
(654, 654)
(654, 654)
(654, 654)
(654, 654)
(654, 654)
(654, 654)
(654, 654)
(654, 654)
(654, 654)
(654, 654)
(654, 654)
(654, 654)
(654, 654)
(654, 654)
(654, 654)
(654, 654)


## WIDC

In [19]:
ds = load_uci(dsn)
ds.with_preprocess(steps=["sample_n", "categorical_only", "remove_constant_columns", "drop_duplicate_rows"],  params_list=[{"n": 1000, "random_state": 0}, {}, {}, {}], in_place=True)


In [4]:
# SUFFIX = "100iter"
SUFFIX = "200iter_wid"
ENCODE = "ordinal_wo_n"
ALSTRATEGY = "widc2"
model_extra_params = { "iterations": 200 }
candidates = ["euclidean", "cosine", "hamming", "smirnov", "goodall1", "iof", "anderberg", "gambaryan"]
for dsn in ["adult"]:
    # if dsn in ["balance-scale"]:
    #     continue
    ds = load_uci(dsn)
    # ds.with_preprocess(steps=["sample_n", "categorical_only", "remove_constant_columns", "drop_duplicate_rows"],  params_list=[{"n": 1000, "random_state": 0}, {}, {}, {}], in_place=True)
    ds.with_preprocess(steps=["sample_n", "remove_constant_columns", "drop_duplicate_rows"],  params_list=[{"n": 1000, "random_state": 0}, {}, {}], in_place=True)

    # ds.with_preprocess(steps=["sample_n", "continuous_to_categorical", "remove_constant_columns", "drop_duplicate_rows"],  params_list=[{"n": 1000, "random_state": 0}, {"encode": ENCODE}, {}, {}], in_place=True)
    ds._data.reset_index(drop=True, inplace=True)
    ds._label.reset_index(drop=True, inplace=True)

    model_name = "catboost"
    columns = ds._data.columns
    for col in ds.info["cat_idx"]:
        ds._data[columns[col]] = pd.Categorical(pd.Categorical(ds._data[columns[col]]).codes.astype("int"))
        # df[column] = pd.Categorical(df[column]).codes
        # ds._data[col] = pd.Categorical(ds._data[col]).codes.astype("int")
    print(ds.info["cat_idx"])
    al_strategy = get_al_strategy(ALSTRATEGY)
    for metric in ["euclidean", "cosine"]:
    # for metric in candidates:
        # ---------------------- IMPORTANT ----------------------------
        cfn = f"{DIST_CACHE}/{dsn}/gambaryan_catonly_wid.npy"
        nfn = f"{DIST_CACHE}/{dsn}/{metric}_numerical.npy"
        # fn = f"{DIST_CACHE}/{dsn}/{metric}_{ENCODE}.npy"
        # fn = f"{DIST_CACHE}/{dsn}/{metric}_catonly_wo_n.npy"
        # print(fn)
        print(cfn, nfn)
        cache_name = f"{dsn.replace('/', '_')}@{model_name}@{ALSTRATEGY}_{metric}+gambaryan_{ENCODE}@{SUFFIX}@x20.pkl"
        if ensure_path(os.path.join(LOCAL_DIR, "ppl_cache_pred", cache_name), False):
            print(f"exp {cache_name} already exists, continue")
            continue
        if not ensure_path(cfn, False) or not ensure_path(nfn, False):
            print(f"{metric} for {dsn} not found in {fn}, skipping ...")
            continue
        print(f"Using cache of {metric} for {dsn} with {ENCODE} encoding")
        # setattr(al_strategy, "dist_cache_path", fn)
        ds = ds.with_strategy(al_strategy, extra_params={"distance_metric": metric, "weighted": True})
        ds.reset()
        # setattr(ds.al_metric, "dist_cache_path", fn)
        setattr(ds.al_metric, "dist_cache_path", (cfn, nfn))

        make_ppl = get_pipeline(model_name)
        ppl = make_ppl(
            model=None,
            # eval_metrics=get_metrics(["roc_auc"]),
            eval_metrics=[],
            seeds=[i for i in range(20)],
            n_times=20,
            dataset=ds,
            cat_idx=ds.info["cat_idx"],
            model_extra_params=model_extra_params
        )
        ppl.start()
        with open(os.path.join(LOCAL_DIR, "ppl_cache_pred", cache_name), "wb") as f:
            pickle.dump(ppl.stats, f)

[1, 3, 5, 6, 7, 8, 9, 13]
examples/dist_cache_1000/adult/gambaryan_catonly_wid.npy examples/dist_cache_1000/adult/euclidean_numerical.npy
Using cache of euclidean for adult with ordinal_wo_n encoding
[WARNING] Start without evaluating metrics


100%|██████████| 20/20 [04:50<00:00, 14.51s/it]


examples/dist_cache_1000/adult/gambaryan_catonly_wid.npy examples/dist_cache_1000/adult/cosine_numerical.npy
Using cache of cosine for adult with ordinal_wo_n encoding
[WARNING] Start without evaluating metrics


100%|██████████| 20/20 [07:19<00:00, 21.98s/it]


In [6]:
ALSTRATEGY = "typiclust"
# for dsn in AVAIL_DATASETS:
SUFFIX = "200iter_numonly_wo_n"
ENCODE = "ordinal_wo_n"
model_extra_params = { "iterations": 200 }
candidates = ["euclidean", "cosine", "hamming", "smirnov", "goodall1", "iof", "anderberg", "gambaryan"]
for dsn in ["adult"]:
    # if dsn in ["balance-scale"]:
    #     continue
    ds = load_uci(dsn)
    # ds.with_preprocess(steps=["sample_n", "categorical_only", "remove_constant_columns", "drop_duplicate_rows"],  params_list=[{"n": 1000, "random_state": 0}, {}, {}, {}], in_place=True)
    ds.with_preprocess(steps=["sample_n", "continuous_only", "remove_constant_columns", "drop_duplicate_rows"],  params_list=[{"n": 1000, "random_state": 0}, {}, {}, {}], in_place=True)
    ds._data.reset_index(drop=True, inplace=True)
    ds._label.reset_index(drop=True, inplace=True)

    model_name = "catboost"
    columns = ds._data.columns
    # for col in ds.info["cat_idx"]:
    #     ds._data[columns[col]] = pd.Categorical(pd.Categorical(ds._data[columns[col]]).codes.astype("int"))
        # df[column] = pd.Categorical(df[column]).codes
        # ds._data[col] = pd.Categorical(ds._data[col]).codes.astype("int")
    print(ds.info["cat_idx"])
    al_strategy = get_al_strategy(ALSTRATEGY)
    for metric in candidates:
    # for metric in ["cosine", "euclidean"]:
        # fn = f"{DIST_CACHE}/{dsn}/{metric}_{ENCODE}.npy"
        fn = f"{DIST_CACHE}/{dsn}/{metric}_numerical.npy"
        print(fn)
        cache_name = f"{dsn.replace('/', '_')}@{model_name}@{ALSTRATEGY}_{metric}_{ENCODE}@{SUFFIX}@x20.pkl"
        # if ensure_path(os.path.join(LOCAL_DIR, "ppl_cache_pred", cache_name), False):
        #     print(f"exp {cache_name} already exists, continue")
        #     continue
        if not ensure_path(fn, False):
            print(f"{metric} for {dsn} not found in {fn}, skipping ...")
            continue
        print(f"Using cache of {metric} for {dsn} with {ENCODE} encoding")
        ds = ds.with_strategy(al_strategy, extra_params={"distance_metric": metric})
        ds.reset()
        setattr(ds.al_metric, "dist_cache_path", fn)
        # setattr(al_strategy, "dist_cache_path", fn)
        # ds = ds.with_strategy(al_strategy, extra_params={"distance_metric": metric})
        make_ppl = get_pipeline(model_name)
        ppl = make_ppl(
            model=None,
            # eval_metrics=get_metrics(["roc_auc"]),
            eval_metrics=[],
            seeds=[i for i in range(20)],
            n_times=20,
            dataset=ds,
            cat_idx=ds.info["cat_idx"],
            model_extra_params=model_extra_params
        )
        ppl.start()
        with open(os.path.join(LOCAL_DIR, "ppl_cache_pred", cache_name), "wb") as f:
            pickle.dump(ppl.stats, f)

[]
examples/dist_cache_1000/adult/euclidean_numerical.npy
Using cache of euclidean for adult with ordinal_wo_n encoding
[WARNING] Start without evaluating metrics


100%|██████████| 20/20 [01:19<00:00,  4.00s/it]


examples/dist_cache_1000/adult/cosine_numerical.npy
Using cache of cosine for adult with ordinal_wo_n encoding
[WARNING] Start without evaluating metrics


100%|██████████| 20/20 [01:20<00:00,  4.03s/it]


examples/dist_cache_1000/adult/hamming_numerical.npy
Using cache of hamming for adult with ordinal_wo_n encoding
[WARNING] Start without evaluating metrics


...

100%|██████████| 20/20 [01:27<00:00,  4.40s/it]


examples/dist_cache_1000/adult/anderberg_numerical.npy
Using cache of anderberg for adult with ordinal_wo_n encoding
[WARNING] Start without evaluating metrics


100%|██████████| 20/20 [01:22<00:00,  4.12s/it]


examples/dist_cache_1000/adult/gambaryan_numerical.npy
Using cache of gambaryan for adult with ordinal_wo_n encoding
[WARNING] Start without evaluating metrics


100%|██████████| 20/20 [01:26<00:00,  4.35s/it]


In [21]:
import matplotlib.pyplot as plt  

def plot(
        dataset_name: str, 
        model_name: str, 
        # strategies: list = None, 
        distance_metrics: list=None, 
        plot_name: str = None,
        strategy: str = "gsx"
):
    metrics_n_times = []
    instances = []
    distance_metrics = distance_metrics or list(distance.AVAIL_DIST_METRICS.keys())
    plot_name = plot_name if plot_name is not None else f"{dataset_name}@{model_name}@{strategy}.png"
    # if ensure_path(os.path.join("./plots/pl_metric", plot_name), False):
    #     print(f"plot {plot_name} already exists, continue")
    #     return
    # for strategy in strategies:
    for metric in distance_metrics:
        stats = None
        cache_name = f"{dataset_name.replace('/', '_')}@{model_name}@{strategy}_{metric}@x20.pkl"
        if not os.path.isfile(os.path.join("ppl_cache", cache_name)):
            print(f"exp {cache_name} does not exist, continue")
            continue
        with open(os.path.join("ppl_cache", cache_name), "rb") as f:
            stats = pickle.load(f)
        metrics_n_times.append([stats[i]["accuracy"] for i in range(len(stats))])
        instances = stats[0]["instances"]
    pl_metric = PLMetric()
    pl_metric.plot("Accuracy", instances, np.array(metrics_n_times), distance_metrics, plot_name=plot_name, cmap=plt.get_cmap("Paired"), dpi=300, title=f"Accuracy {dataset_name}@{model_name}@{strategy}", errorbar="std")

In [24]:
def plot_by_strategies(
        dataset_name: str, 
        model_name: str, 
        distance_metric: str, 
        plot_name: str = None,
        strategies: list = None
):
    metrics_n_times = []
    instances = []
    strategies = strategies or ["gsx", "typiclust"]
    plot_name = plot_name if plot_name is not None else f"vs/{dataset_name}@{model_name}@{distance_metric}@{'_'.join(strategies)}.png"
    print("plotting ", plot_name)
    # if ensure_path(os.path.join("./plots/pl_metric", plot_name), False):
    #     print(f"plot {plot_name} already exists, continue")
    #     return
    # for strategy in strategies:
    for strategy in strategies:
        stats = None
        cache_name = f"{dataset_name.replace('/', '_')}@{model_name}@{strategy}_{distance_metric}@x20.pkl"
        if not os.path.isfile(os.path.join("ppl_cache_bk", cache_name)):
            print(f"exp {cache_name} does not exist, continue")
            return
        with open(os.path.join("ppl_cache_bk", cache_name), "rb") as f:
            stats = pickle.load(f)
        # print(len(stats))
        metrics_n_times.append(np.array([stats[i]["accuracy"] for i in range(len(stats))]))
        instances = stats[0]["instances"]
    pl_metric = PLMetric()
    pl_metric.plot("Accuracy", instances, np.array(metrics_n_times), strategies, plot_name=plot_name, cmap=plt.get_cmap("Paired"), dpi=300, errorbar="std", title=f"Accuracy {dataset_name}@{model_name}@{distance_metric}")

In [25]:
# print by strategies
for dsn in AVAIL_DATASETS:
    for m in list(distance.AVAIL_DIST_METRICS.keys()):
        plot_by_strategies(dsn, "catboost", m, strategies=["gsx", "typiclust"])

plotting  vs/iris@catboost@euclidean@gsx_typiclust.png
plotting  vs/iris@catboost@cosine@gsx_typiclust.png
plotting  vs/iris@catboost@overlap@gsx_typiclust.png
plotting  vs/iris@catboost@eskin@gsx_typiclust.png
plotting  vs/iris@catboost@iof@gsx_typiclust.png
plotting  vs/iris@catboost@of@gsx_typiclust.png
plotting  vs/iris@catboost@lin@gsx_typiclust.png
plotting  vs/iris@catboost@lin1@gsx_typiclust.png
plotting  vs/iris@catboost@goodall1@gsx_typiclust.png
plotting  vs/iris@catboost@goodall2@gsx_typiclust.png
plotting  vs/iris@catboost@goodall3@gsx_typiclust.png
plotting  vs/iris@catboost@goodall4@gsx_typiclust.png
plotting  vs/iris@catboost@smirnov@gsx_typiclust.png
plotting  vs/iris@catboost@gambaryan@gsx_typiclust.png
plotting  vs/iris@catboost@burnaby@gsx_typiclust.png
plotting  vs/iris@catboost@anderberg@gsx_typiclust.png
plotting  vs/adult@catboost@euclidean@gsx_typiclust.png
plotting  vs/adult@catboost@cosine@gsx_typiclust.png
plotting  vs/adult@catboost@overlap@gsx_typiclust.pn

In [ ]:
# print by distance metrics | TypiClust
for dsn in AVAIL_DATASETS:
    plot(dsn, "catboost",distance_metrics=list(distance.AVAIL_DIST_METRICS.keys()), strategy="typiclust")

In [ ]:
# print by distance metrics | GSX
for dsn in AVAIL_DATASETS:
    plot(dsn, "catboost", distance_metrics=list(distance.AVAIL_DIST_METRICS.keys()), strategy="gsx")

In [14]:
for m in list(distance.AVAIL_DIST_METRICS.keys()):
    plot("balance-scale", "catboost", [m, ], plot_name=f"test-{m}.png")

# Test TypiClust

In [3]:
adult = load_uci("adult")
adult.with_preprocess(steps=["sample_n", "continuous_to_categorical", "remove_constant_columns", "drop_duplicate_rows"],  params_list=[{"n": 1000, "random_state": 0}, {"encode": "ordinal"}, {}, {}], in_place=True)
print(len(adult._data))
print(len(adult._data.drop_duplicates()))

971
971


In [4]:

import pandas as pd 
  
# Create a dictionary of students 
students = { 
    'Name': ['Lisa', 'Kate', 'Ben', 'Kim', 'Josh', 
             'Alex', 'Evan', 'Greg', 'Sam', 'Ella'], 
    'ID': ['001', '002', '003', '004', '005', '006',  
           '007', '008', '009', '010'], 
    'Grade': ['A', 'A', 'C', 'B', 'B', 'B', 'C',  
              'A', 'A', 'A'], 
    
    'Category': [2, 3, 1, 3, 2, 3, 3, 1, 2, 1] 
} 
  
# Create dataframe from students dictionary 
df = pd.DataFrame(students) 

In [56]:
ds = load_uci(dsn)
ds.with_preprocess(steps=["sample_n", "categorical_only", "remove_constant_columns", "drop_duplicate_rows"],  params_list=[{"n": 1000, "random_state": 0}, {}, {}, {}], in_place=True)
al_strategy = get_al_strategy("random")
ds = ds.with_strategy(al_strategy, extra_params={"distance_metric": metric})
ds.bind_params({"current_stat": {"snapshot": []}})
for s in ds:
    first = s
    break

In [63]:
first[0].index, first2[0].index

(Index([19, 757, 725, 197, 435, 766, 60, 530, 752, 673], dtype='int64'),
 Index([19, 757, 725, 197, 435, 766, 60, 530, 752, 673], dtype='int64'))

In [15]:
ds = load_uci(dsn)
ds.with_preprocess(steps=["sample_n", "categorical_only", "remove_constant_columns", "drop_duplicate_rows"],  params_list=[{"n": 1000, "random_state": 0}, {}, {}, {}], in_place=True)
ds._data.cp
# al_strategy = get_al_strategy("typiclust")
# fn = f"{DIST_CACHE}/{dsn}/iof_catonly.npy"
# setattr(al_strategy, "dist_cache_path", fn)
# ds = ds.with_strategy(al_strategy, extra_params={"distance_metric": metric})
# ds.bind_params({"current_stat": {"snapshot": []}})
# for s in ds:
#     first2 = s
#     break

0      1.0
1      4.0
2      3.0
3      4.0
4      2.0
      ... 
279    2.0
280    4.0
282    3.0
287    4.0
296    4.0
Name: cp, Length: 126, dtype: category
Categories (4, float64): [1.0, 2.0, 3.0, 4.0]

In [7]:
LOCAL_DIR = "app/comp_dist"
SUFFIX = "200iter_numonly_wo_n"
ENCODE = "ordinal_wo_n"
model_extra_params = { "iterations": 200 }
ALSTRATEGY = "typiclust"
# for dsn in AVAIL_DATASETS:
for dsn in ["adult"]:
    ds = load_uci(dsn)
    ds.with_preprocess(steps=["sample_n", "continuous_only", "remove_constant_columns", "drop_duplicate_rows"],  params_list=[{"n": 1000, "random_state": 0}, {}, {}, {}], in_place=True)
    # ds.with_preprocess(steps=["sample_n", "remove_constant_columns", "drop_duplicate_rows"],  params_list=[{"n": 1000, "random_state": 0}, {}, {}], in_place=True)
    ds._data.reset_index(drop=True, inplace=True)
    ds._label.reset_index(drop=True, inplace=True)

    model_name = "catboost"
    # for col in ds._data.columns:
    #     # df[column] = pd.Categorical(df[column]).codes
    #     ds._data[col] = pd.Categorical(ds._data[col]).codes.astype("int")
    # columns = ds._data.columns
    # for col in ds.info["cat_idx"]:
    #     ds._data[columns[col]] = pd.Categorical(pd.Categorical(ds._data[columns[col]]).codes.astype("int"))
    al_strategy = get_al_strategy("random")
    
    for metric in ["cosine"]:
        # fn = f"{DIST_CACHE}/{dsn}/{metric}_{ENCODE}.npy"
        fn = f"{DIST_CACHE}/{dsn}/{metric}_catonly_wo_n.npy"
        cache_name = f"{dsn.replace('/', '_')}@{model_name}@random_{metric}_{ENCODE}@{SUFFIX}@x20.pkl"
        print(os.path.join(LOCAL_DIR, "ppl_cache_pred", cache_name))
        # if ensure_path(os.path.join(LOCAL_DIR, "ppl_cache_pred", cache_name), False):
        #     print(f"exp {cache_name} already exists, continue")
        #     continue
        if not ensure_path(fn, False):
            print(f"{metric} for {dsn} not found({fn}), skipping ...")
            continue
        print(f"Using cache of {metric} for {dsn}")
        # ds = ds.with_strategy(al_strategy, extra_params={"distance_metric": metric})
        ds = ds.with_strategy(al_strategy, extra_params={"distance_metric": metric})
        ds.reset()
        setattr(ds.al_metric, "dist_cache_path", fn)
        make_ppl = get_pipeline(model_name)
        ppl = make_ppl(
            model=None,
            eval_metrics=[],
            seeds=[i for i in range(20)],
            n_times=20,
            dataset=ds,
            cat_idx=ds.info["cat_idx"],
            model_extra_params=model_extra_params
        )
        ppl.start()
        
        with open(os.path.join(LOCAL_DIR, "ppl_cache_pred", cache_name), "wb") as f:
            pickle.dump(ppl.stats, f)

app/comp_dist/ppl_cache_pred/adult@catboost@random_cosine_ordinal_wo_n@200iter_numonly_wo_n@x20.pkl
Using cache of cosine for adult
[WARNING] Start without evaluating metrics


100%|██████████| 20/20 [01:07<00:00,  3.37s/it]


In [67]:
with open('app/comp_dist/ppl_cache_pred/adult@catboost@typiclust_euclidean_ordinal@100iter_catonly@x20.pkl', "rb") as f:
    st = pickle.load(f)
with open('app/comp_dist/ppl_cache_pred/adult@catboost@random_cosine_ordinal@100iter_catonly@x20.pkl', "rb") as f:
    st2 = pickle.load(f)
with open('app/comp_dist/ppl_cache_pred/adult@catboost@typiclust_of_ordinal@100iter_catonly@x20.pkl', "rb") as f:
    st3 = pickle.load(f)

In [69]:
st2[0]["predictions"][0]

array(['<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K',
       '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K',
       '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K',
       '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K',
       '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K',
       '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K',
       '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K',
       '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K',
       '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K',
       '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K',
       '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K',
       '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K',
       '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K',
       '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K',
       '<=50K', '<=5

In [6]:
dataset = load_uci("iris")

dataset.with_preprocess(
    steps=[
        "sample_n", 
        "continuous_to_categorical", 
        "remove_constant_columns", 
        "drop_duplicate_rows"
    ],  
    params_list=[
        {"n": 1000, "random_state": 0}, 
        {"encode": "ordinal"}, 
        {}, 
        {}
    ], in_place=True)

al_strategy = get_al_strategy("random")
dataset = dataset.with_strategy(
    al_strategy, 
    extra_params={"distance_metric": "cosine"})

from catboost import CatBoostClassifier

# model = CatBoostClassifier(task_type="GPU", devices="0:1")
# model = CatBoostClassifier()
# model.fit(dataset._data, dataset._label, cat_features=dataset.info["cat_idx"])
model_extra_params = {"iterations": 1000}
ds = load_uci("iris")
make_ppl = get_pipeline("catboost")
ppl = make_ppl(
    model=None,
    eval_metrics=get_metrics([]),
    seeds=[i for i in range(20)],
    n_times=20,
    dataset=ds,
    cat_idx=ds.info["cat_idx"],
    model_extra_params=model_extra_params
)

In [10]:
from sklearn.metrics import get_scorer_names
[i for i in get_scorer_names() if i.startswith("roc")]

['roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_weighted']